In [1]:
# Python Script to Extract tweets of a 
# particular Hashtag using Tweepy and Pandas

# import modules
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import datetime
import tweepy
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\axbae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\axbae\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Combining dataframes

In [13]:
df1=pd.read_csv('df1.csv')
df2=pd.read_csv('df2.csv')
df3=pd.read_csv('df3.csv')

obs = [df1, df2, df3]

data_set = pd.concat(obs)
data_set = data_set.reset_index(drop = True)
#data_set

### Cleaning the DataFrame

#### Dropping & Creating Columns 

In [5]:
#Dropping collumns
data_set1 = data_set.drop(['Unnamed: 0','username', 'description', 'followers', 'following'], axis=1).reset_index(drop = True)

#Creating collumn for hashtags
data_set1['hashtags'] = data_set1['text'].apply(lambda x: re.findall(r'#(\w+)', x))

#Calling text column
tweets = data_set1['text'].values

#### Emojis & Emoticons

In [ ]:
#https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python

In [4]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
         "]+", flags=re.UNICODE)

#### Cleaning with Regex Expressions

In [6]:
import numpy as np
wnl = nltk.WordNetLemmatizer()

def clean_tweet(tweet):
    if type(tweet) == np.float64:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) 
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('[.,*?:]',' ', temp)
    temp = re.sub('\v', ' ', temp)
    temp = re.sub("[\\n]",' ', temp)
    temp = re.sub('1/n',' ', temp)
    temp = re.sub('2/n',' ', temp)
    temp = re.sub('3/n',' ', temp)
    temp = re.sub('4/n',' ', temp)
    temp = re.sub('5/n',' ', temp)
    temp = re.sub('6/n',' ', temp)
    temp = re.sub('7/n',' ', temp)
    temp = re.sub('8/n',' ', temp)
    temp = re.sub('-','', temp) 
    temp = re.sub('%','', temp) 
    temp = re.sub('["]','', temp)
    temp = re.sub("[']","", temp)
    temp = re.sub('[0-9]', ' ', temp)
    temp = re.sub('\+', ' ', temp)
   
    temp = emoji_pattern.sub(r'', temp)
    return temp

#### Sentiment Analysis using Afinn (danish)

In [7]:
#importer dansk som sprog
#pip install afinn
from afinn import Afinn
afinn = Afinn(language='da')

#### Looping over rows in "text" column: cleaning and assigning sentiment score

In [12]:

tweetser = [None for i in range(len(data_set1))]

score = [None for i in range(len(data_set1))]
for i in range(len(data_set1)):  
    tweetser[i] = clean_tweet(data_set1.text.values[i])
    score[i]= afinn.score(tweetser[i])
    
#count reetweets
#Counter(tweetser).most_common()

#### Inserting new columns into the original data set

In [9]:
data_set1['Clean Text'] = tweetser
data_set1['Sentiment'] = score
final_set = data_set1.drop(['text'], axis=1).reset_index(drop = True)
#final_set

### Term Frequency 

#### Deleting ' , in strings

In [ ]:
clean_data = [clean_tweet(tweets) for tweets in tweets]

c = str(clean_data) #converting the clean_data from a list to a string
a = c.replace("'", "")
b = a.replace(",", "")

#### Removing stopwords

In [ ]:
#Remove danish stopWords
stopWords = set(stopwords.words('danish'))
sent_sw_removed = [i for i in nltk.word_tokenize(b.lower()) if i not in stopWords]

#### Counting most frequent words

In [ ]:
from collections import Counter

tæl = Counter(sent_sw_removed).most_common()

#### Ordering in tabel

In [ ]:
counter_tabel=pd.DataFrame(tæl)
counter_tabel.columns = ['Ord', 'Antal']

counter_tabel